In [602]:
#Trial Based Data Pre-Processing

from scipy.io import loadmat
import mne 
mne.set_log_level('error')
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt
TrialData = loadmat('P08.mat')
type(TrialData)
TrialData.keys()



dict_keys(['__header__', '__version__', '__globals__', 'bciexp', 'subject'])

In [604]:
#filtering with MNE lib

BCI = TrialData['bciexp']
EEGData = BCI['data'][0,0] #nChannles x nSamples x nTrials
#print(BCI['label'])
sfreq = float(BCI['srate'][0, 0])  # Sampling frequency
n_channels = EEGData.shape[0]
#ch_names = BCI['label'][0,0] # Dummy channel names, replace as needed

ch_names = ['P7', 'P5', 'P3', 'P4','P6', 'P8', 'PO7', 'PO3', 'PO4', 'PO8', 'O1', 'O2']
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')

# Reshape the data to (nTrials, nChannels, nSamples) as required by MNE
EEGData_mne = np.transpose(EEGData, (2, 0, 1))

# Create MNE EpochsArray object
epochs = mne.EpochsArray(EEGData_mne, info)

# Now apply the filter using MNE's filter function
filtered_epochs = epochs.filter(l_freq=1, h_freq=15, fir_design='firwin', filter_length='auto')
#filtered_epochs = filtered_epochs.resample(64)
# To visualize or further process filtered data
filtered_data = filtered_epochs.get_data()
print(filtered_data.shape)  # This will still be (nTrials, nChannels, nSamples)




C:\Users\thake\AppData\Local\Temp\ipykernel_20000\1316363568.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sfreq = float(BCI['srate'][0, 0])  # Sampling frequency


(144, 12, 4000)


In [606]:
stim_id = BCI['stim'][0,0]
intention = BCI['intention'][0,0]
Expec = BCI['expected'][0,0]
Target = BCI['targetside'][0,0]

# Assuming `EEG_data` is your filtered and resampled EEG data
# stim, intention, targetside are your other fields
n_trials = filtered_data.shape[0]
print(Target)



[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [346]:
#Re-epoching of data based on stim_ch 1 & 2

# Parameters for re-epoching
stim_id = BCI['stim'][0,0]
tmin, tmax = -0.2, 0.8  # Time window around the stimulus (200 ms before, 800 ms after)
sfreq = 250  # Sampling frequency
n_trials = filtered_data.shape[0]  # Total number of trials
condition_labels = []  # To store labels for each re-epoch based on the four conditions

# Create an empty list to hold re-epoch data
re_epoch_data = []

# Loop through each trial
for trial_id in range(n_trials):
    EEG_trial = filtered_data[trial_id, :, :]  # Extract the EEG data for this trial

    # Get the stimulus onsets and target side for this trial
    stim_ch1 = stim_id[0, :, trial_id]  # Stimulus channel 1
    stim_ch2 = stim_id[1, :, trial_id]  # Stimulus channel 2
    targetside_trial = Target[0, :, trial_id]  # Target side for this trial

    # Find the indices (sample points) where stimulus onset occurs
    stim_onsets_ch1 = np.where(stim_ch1 == 1)[0]  # Stimulus onset for Ch1
    stim_onsets_ch2 = np.where(stim_ch2 == 1)[0]  # Stimulus onset for Ch2

    # Combine stimulus onsets and create a list of (onset, stim_type) for labeling
    all_stimuli_onsets = [(onset, 1) for onset in stim_onsets_ch1] + \
                         [(onset, 2) for onset in stim_onsets_ch2]

    # Sort all stimuli onsets by time (ascending order)
    all_stimuli_onsets.sort(key=lambda x: x[0])

    # Loop through each stimulus onset to re-epoch the data
    for onset, stim_type in all_stimuli_onsets:
        # Convert onset to seconds
        onset_sec = onset / sfreq

        # Find the sample indices for the time window around the stimulus onset
        start_sample = int(onset + tmin * sfreq)
        end_sample = int(onset + tmax * sfreq)

        # Ensure indices are within bounds
        if start_sample >= 0 and end_sample < EEG_trial.shape[1]:
            # Extract the EEG segment for this stimulus onset
            re_epoch = EEG_trial[:, start_sample:end_sample]

            # Append to re-epoch data
            re_epoch_data.append(re_epoch)

           # Append the stimulus type label (1 for Ch1, 2 for Ch2)
            condition_labels.append(stim_type)

# Convert the re-epoch data list to a NumPy array (n_epochs, n_channels, n_samples)
re_epoch_data = np.array(re_epoch_data)
print(re_epoch_data.shape)
# Create a new MNE EpochsArray object with re-epoch data
re_epochs = mne.EpochsArray(re_epoch_data, info)
re_epochs = re_epochs.get_data()

print("Condition labels:", condition_labels)

(1440, 12, 250)
Condition labels: [1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1,

In [348]:
#Plotting the Trial Based Data

# Parameters for plotting
trial_id = 3  # Select the trial you want to plot
channel_id = 3  # Select the specific channel you want to plot (e.g., 3rd channel)
tmin, tmax = -0.2, 0.8  # Time window used in re-epoching
sfreq = 250  # Sampling frequency

# Extract the single trial and single channel data
single_trial = filtered_data[trial_id, channel_id, :]  # Extract a specific trial and channel

# Generate the time vector corresponding to tmin and tmax
n_samples = single_trial.shape[0]
time = np.linspace(tmin, tmax, n_samples)  # Time array for x-axis (in seconds)

# Baseline correction: Subtract the mean of the pre-stimulus period (e.g., tmin to 0)
baseline_start = int((tmin - tmin) * sfreq)  # Index of the start of the baseline (relative to tmin)
baseline_end = int((0 - tmin) * sfreq)  # Index of the end of the baseline (0 seconds)
baseline_mean = np.mean(single_trial[baseline_start:baseline_end])  # Calculate the baseline mean

# Apply baseline correction
single_trial_baseline_corrected = single_trial - baseline_mean

# Plot the single channel data (with baseline correction)
plt.figure(figsize=(10, 6))
plt.plot(time, single_trial_baseline_corrected, label=ch_names[channel_id], color='b')

# Add labels and title
plt.xlabel('Time (seconds)')
plt.ylabel('EEG Amplitude (µV)')
plt.title(f'Trial {trial_id + 1} - Channel {ch_names[channel_id]} (Baseline Corrected)')

# Add a grid for better visualization
plt.grid(True)

# Display the plot
plt.tight_layout()
plt.show()

In [350]:
#averaging the every 10 trials based on stim channel 1 & 2
import numpy as np

# Initialize lists to store the averaged data
avg_label_1_per_group = []
avg_label_2_per_group = []

# Number of trials per group
group_size = 10

# Loop over the data in chunks of 10 trials
for i in range(0, len(re_epoch_data), group_size):
    # Extract the current group of 10 trials
    group_data = re_epoch_data[i:i + group_size]
    group_labels = condition_labels[i:i + group_size]
    
    # Check if we have exactly 10 trials in this group
    if len(group_data) < group_size:
        continue  # Skip if we have fewer than 10 trials at the end
    
    # Separate trials by label within the group
    trials_label_1 = [group_data[j] for j in range(group_size) if group_labels[j] == 1]
    trials_label_2 = [group_data[j] for j in range(group_size) if group_labels[j] == 2]
    
    # Ensure we have 5 trials of each label before averaging
    if len(trials_label_1) == 5 and len(trials_label_2) == 5:
        # Compute the averages for label "1" and label "2" trials within this group
        avg_label_1 = np.mean(trials_label_1, axis=0)
        avg_label_2 = np.mean(trials_label_2, axis=0)
        
        # Append to lists
        avg_label_1_per_group.append(avg_label_1)
        avg_label_2_per_group.append(avg_label_2)
    else:
        print(f"Warning: Skipped group {i//group_size + 1} due to imbalance in labels")

# Convert the lists to NumPy arrays
avg_label_1_per_group = np.array(avg_label_1_per_group)  # Shape (144, n_channels, n_samples)
avg_label_2_per_group = np.array(avg_label_2_per_group)  # Shape (144, n_channels, n_samples)




print("Averaged data shape for label 1:", avg_label_1_per_group.shape)
print("Averaged data shape for label 2:", avg_label_2_per_group.shape)

Averaged data shape for label 1: (144, 12, 250)
Averaged data shape for label 2: (144, 12, 250)


In [352]:
# Z-Score Normalization
def z_score_normalize(data):
    # data shape: (n_trials, n_channels, n_samples)
    mean = np.mean(data, axis=(1, 2), keepdims=True)  # Compute mean for each trial
    std = np.std(data, axis=(1, 2), keepdims=True)    # Compute std for each trial
    normalized_data = (data - mean) / std            # Z-score normalization
    return normalized_data

# Normalize your datasets
avg_label_1_normalized = z_score_normalize(avg_label_1_per_group)
avg_label_2_normalized = z_score_normalize(avg_label_2_per_group)

# Check shapes
print("Normalized data shape (label 1):", avg_label_1_normalized.shape)
print("Normalized data shape (label 2):", avg_label_2_normalized.shape)


Normalized data shape (label 1): (144, 12, 250)
Normalized data shape (label 2): (144, 12, 250)


In [354]:
from mne.decoding import CSP
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

# Combine data and labels for CSP
X = np.concatenate([avg_label_1_per_group, avg_label_2_per_group], axis=0)  # Shape: (288, n_channels, n_samples)
y = np.concatenate([np.ones(len(avg_label_1_per_group)), np.zeros(len(avg_label_2_per_group))])  # Labels

# Reshape data if needed (already correct format)
print("Shape of X:", X.shape)  # Should be (n_trials, n_channels, n_samples)
print("Shape of y:", y.shape)  # Should be (n_trials,)

# Initialize CSP
csp = CSP(n_components=12, reg=None, log=True, cov_est='concat')  # 4 components for simplicity

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fit CSP on training data
csp.fit(X_train, y_train)

# Transform the data using CSP
X_train_csp = csp.transform(X_train)  # CSP-transformed features for training
X_test_csp = csp.transform(X_test)    # CSP-transformed features for testing

# Train an SVM classifier on CSP-transformed features
clf = SVC(kernel='linear', random_state=42)
clf.fit(X_train_csp, y_train)

# Test the classifier
y_pred = clf.predict(X_test_csp)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Accuracy with CSP: {accuracy:.2f}")


Shape of X: (288, 12, 250)
Shape of y: (288,)
Classification Accuracy with CSP: 0.72


In [112]:
#plotting the averaged data sequence and channel wise

import matplotlib.pyplot as plt
import numpy as np

# Parameters
sequence_id = 55  # Choose a specific sequence (e.g., the first sequence out of 144)
channel_id = 8  # Choose a specific channel to plot
tmin, tmax = -0.2, 0.8  # Time window
sfreq = 250  # Sampling frequency
time = np.linspace(tmin, tmax, avg_label_1_per_group.shape[2])  # Time vector for x-axis

# Plot for the chosen sequence and channel
plt.figure(figsize=(10, 6))

# Plot for label 1 (average of 5 trials with label 1 in the selected sequence)
plt.plot(time, avg_label_1_normalized[sequence_id, channel_id, :], label='Label 1 Avg', color='b')

# Plot for label 2 (average of 5 trials with label 2 in the selected sequence)
plt.plot(time, avg_label_2_normalized[sequence_id, channel_id, :], label='Label 2 Avg', color='r')

# Add labels and title
plt.xlabel('Time (seconds)')
plt.ylabel('EEG Amplitude (µV)')
plt.title(f'Averaged EEG Signal - Sequence {sequence_id + 1} - Channel {channel_id}')
plt.legend()
plt.grid(True)

# Display the plot
plt.tight_layout()
plt.show()


In [356]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Assume avg_label_1_per_group and avg_label_2_per_group are (144, n_channels, n_samples)
# Prepare data and labels
data = np.concatenate([avg_label_1_per_group, avg_label_2_per_group], axis=0)  # (288, n_channels, n_samples)
labels = np.array([1] * 144 + [2] * 144)  # Labels for each sample (1 for avg_label_1, 2 for avg_label_2)

# Flatten each sequence for simplicity (you can also extract features as needed)
data_flat = data.reshape(data.shape[0], -1)  # Reshape to (288, n_channels * n_samples)

# Set up cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

# Cross-validation loop
for train_index, test_index in kf.split(data_flat):
    X_train, X_test = data_flat[train_index], data_flat[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    # Initialize and train SVM classifier
    clf = SVC(kernel='linear')  # Choose kernel type as appropriate
    clf.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f'Fold accuracy: {accuracy * 100:.2f}%')
# Print average cross-validation accuracy
print(f'Cross-Validation Accuracy:, {np.mean(accuracies) * 100:.2f}%')
print(data.shape)

Fold accuracy: 98.28%
Fold accuracy: 94.83%
Fold accuracy: 94.83%
Fold accuracy: 96.49%
Fold accuracy: 96.49%
Cross-Validation Accuracy:, 96.18%
(288, 12, 250)


In [618]:
# Parameters for re-epoching based on target side 1 & -1
tmin, tmax = 0.2, 0.8  # Time window around the stimulus 
sfreq = 250  # Sampling frequency
n_trials = filtered_data.shape[0]  # Total number of trials
target_side_labels = []  # To store labels for each re-epoch based on target side (1 for left, -1 for right)

# Create an empty list to hold re-epoch data
re_epoch_data = []

# Loop through each trial
for trial_id in range(n_trials):
    EEG_trial = filtered_data[trial_id, :, :]  # Extract the EEG data for this trial

    # Get the stimulus onsets and target side for this trial
    stim_ch1 = stim_id[0, :, trial_id]  # Stimulus channel 1
    stim_ch2 = stim_id[1, :, trial_id]  # Stimulus channel 2
    targetside_trial = Target[0, :, trial_id]  # Target side for this trial

    # Find the indices (sample points) where stimulus onset occurs
    stim_onsets_ch1 = np.where(stim_ch1 == 1)[0]  # Stimulus onset for Ch1
    stim_onsets_ch2 = np.where(stim_ch2 == 1)[0]  # Stimulus onset for Ch2

    # Combine stimulus onsets and create a list of (onset, stim_type)
    all_stimuli_onsets = [(onset, 1) for onset in stim_onsets_ch1] + \
                         [(onset, 2) for onset in stim_onsets_ch2]

    # Sort all stimuli onsets by time (ascending order)
    all_stimuli_onsets.sort(key=lambda x: x[0])

    # Loop through each stimulus onset to re-epoch the data
    for onset, stim_type in all_stimuli_onsets:
        # Convert onset to seconds
        onset_sec = onset / sfreq

        # Find the sample indices for the time window around the stimulus onset
        start_sample = int(onset + tmin * sfreq)
        end_sample = int(onset + tmax * sfreq)

        # Ensure indices are within bounds
        if start_sample >= 0 and end_sample < EEG_trial.shape[1]:
            # Extract the EEG segment for this stimulus onset
            re_epoch = EEG_trial[:, start_sample:end_sample]

            # Append to re-epoch data
            re_epoch_data.append(re_epoch)

            # Determine the target side at this stimulus onset
            targetside_value = targetside_trial[onset]

            # Assign labels directly based on target side
            if targetside_value == 1:  # Target is on the left
                target_side_labels.append(1)
            elif targetside_value == -1:  # Target is on the right
                target_side_labels.append(-1)

# Convert the re-epoch data list to a NumPy array (n_epochs, n_channels, n_samples)
re_epoch_data2 = np.array(re_epoch_data)
print("Re-epoch data shape:", re_epoch_data2.shape)  # Should be (1440, n_channels, n_samples)

# Convert target side labels to a NumPy array
target_side_labels = np.array(target_side_labels)
print("Target side labels shape:", target_side_labels.shape)  # Should be (1440,)

# Verify the distribution of labels
print("Label counts (1 for left, -1 for right):", np.unique(target_side_labels, return_counts=True))


Re-epoch data shape: (1440, 12, 150)
Target side labels shape: (1440,)
Label counts (1 for left, -1 for right): (array([-1,  1]), array([720, 720], dtype=int64))


In [620]:
#averaging the every 10 trials based on target side 1 & -1
import numpy as np

# Initialize lists to store the averaged data
avg_label_P1_per_group = []
avg_label_M1_per_group = []

# Number of trials per group
group_size = 10

# Loop over the data in chunks of 10 trials
for i in range(0, len(re_epoch_data2), group_size):
    # Extract the current group of 10 trials
    group_data = re_epoch_data2[i:i + group_size]
    group_labels = target_side_labels[i:i + group_size]
    
    # Check if we have exactly 10 trials in this group
    if len(group_data) < group_size:
        continue  # Skip if we have fewer than 10 trials at the end
    
    # Separate trials by label within the group
    trials_label_P1 = [group_data[j] for j in range(group_size) if group_labels[j] == 1]
    trials_label_M1 = [group_data[j] for j in range(group_size) if group_labels[j] == -1]
    
    # Ensure we have 5 trials of each label before averaging
    if len(trials_label_P1) == 5 and len(trials_label_M1) == 5:
        # Compute the averages for label "1" and label "2" trials within this group
        avg_label_P1 = np.mean(trials_label_P1, axis=0)
        avg_label_M1 = np.mean(trials_label_M1, axis=0)
        
        # Append to lists
        avg_label_P1_per_group.append(avg_label_P1)
        avg_label_M1_per_group.append(avg_label_M1)
    else:
        print(f"Warning: Skipped group {i//group_size + 1} due to imbalance in labels")

# Convert the lists to NumPy arrays
avg_label_P1_per_group = np.array(avg_label_P1_per_group)  # Shape (144, n_channels, n_samples)
avg_label_M1_per_group = np.array(avg_label_M1_per_group)  # Shape (144, n_channels, n_samples)




print("Averaged data shape for label 1:", avg_label_P1_per_group.shape)
print("Averaged data shape for label 2:", avg_label_M1_per_group.shape)

Averaged data shape for label 1: (144, 12, 150)
Averaged data shape for label 2: (144, 12, 150)


In [622]:
#classification of target side based epochs by CSP
from mne.decoding import CSP
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

# Combine data and labels for CSP
X = np.concatenate([avg_label_P1_per_group, avg_label_M1_per_group], axis=0)  # Shape: (288, n_channels, n_samples)
y = np.concatenate([np.ones(len(avg_label_P1_per_group)), np.zeros(len(avg_label_M1_per_group))])  # Labels

# Reshape data if needed (already correct format)
print("Shape of X:", X.shape)  # Should be (n_trials, n_channels, n_samples)
print("Shape of y:", y.shape)  # Should be (n_trials,)

# Initialize CSP
csp = CSP(n_components=4, reg=None, log=True, cov_est='concat')  # 4 components for simplicity

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fit CSP on training data
csp.fit(X_train, y_train)

# Transform the data using CSP
X_train_csp = csp.transform(X_train)  # CSP-transformed features for training
X_test_csp = csp.transform(X_test)    # CSP-transformed features for testing

# Train an SVM classifier on CSP-transformed features
clf = SVC(kernel='linear', random_state=42)
clf.fit(X_train_csp, y_train)

# Test the classifier
y_pred = clf.predict(X_test_csp)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Accuracy with CSP: {accuracy:.2f}")


Shape of X: (288, 12, 150)
Shape of y: (288,)
Classification Accuracy with CSP: 0.71


In [624]:
#CCA_transformation of data with target side 1 & -1

import numpy as np
from sklearn.cross_decomposition import CCA

# Step 1: Prepare EEG Data (X)
# Flatten EEG data (X) across epochs and time points
n_epochs, n_channels, n_samples = re_epoch_data2.shape
X = re_epoch_data2.transpose(0, 2, 1).reshape(n_epochs * n_samples, n_channels)  # Shape: (1440 * 225, 12)

# Step 2: Construct Target Matrix (Y)
# Create identity matrices for left (1) and right (-1) targets
Y = np.zeros((n_epochs * n_samples, 1))
for i, label in enumerate(target_side_labels):  # target_side_labels has shape (1440,)
    start_idx = i * n_samples
    end_idx = (i + 1) * n_samples
    if label == 1:  # Left target
        Y[start_idx:end_idx, 0] = 1 * np.eye(n_samples)[:, 0]  # Positive identity for left
    elif label == -1:  # Right target
        Y[start_idx:end_idx, 0] = -1 * np.eye(n_samples)[:, 0]  # Negative identity for right

# Step 3: Apply Canonical Correlation Analysis (CCA)
cca = CCA(n_components=1)  # One canonical component
cca.fit(X, Y)

# Step 4: Transform Data Using CCA
X_c, Y_c = cca.transform(X, Y)

# Step 5: Compute Canonical Correlation
correlation = np.corrcoef(X_c[:, 0], Y_c[:, 0])[0, 1]
print("Canonical Correlation:", correlation)

# Step 6: Extract Weight Vectors
a = cca.x_weights_  # Spatial filter (weights for EEG data)
b = cca.y_weights_  # Weights for target matrix
print("Spatial Filter (a):", a)
print("Target Weights (b):", b)

# Step 7: Canonical Components
u = X @ a  # EEG-derived time series
v = Y @ b  # Target-derived time series

# Classification Based on Sign of Canonical Correlation
predictions = np.sign(u.flatten())  # Predict left (1) or right (-1)
true_labels = np.repeat(target_side_labels, n_samples)  # Expand target labels
accuracy = np.mean(predictions == true_labels)
print("Classification Accuracy:", accuracy)


Canonical Correlation: 0.025905066377166313
Spatial Filter (a): [[-0.20452624]
 [ 0.02462186]
 [-0.03272652]
 [-0.05137434]
 [ 0.2187501 ]
 [-0.26258401]
 [ 0.5134056 ]
 [ 0.10543234]
 [-0.43299972]
 [-0.07916486]
 [-0.30657381]
 [ 0.52402847]]
Target Weights (b): [[-1.]]
Classification Accuracy: 0.5059166666666667


In [626]:
#k-fold CV with SVM for labels with target side 1 & -1
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Assume avg_label_1_per_group and avg_label_2_per_group are (144, n_channels, n_samples)
# Prepare data and labels
data = np.concatenate([avg_label_P1_per_group, avg_label_M1_per_group], axis=0)  # (288, n_channels, n_samples)
labels = np.array([1] * 144 + [-1] * 144)  # Labels for each sample (1 for avg_label_P1, -1 for avg_label_M1)

# Flatten each sequence for simplicity (you can also extract features as needed)
data_flat = data.reshape(data.shape[0], -1)  # Reshape to (288, n_channels * n_samples)

# Set up cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

# Cross-validation loop
for train_index, test_index in kf.split(data_flat):
    X_train, X_test = data_flat[train_index], data_flat[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    # Initialize and train SVM classifier
    clf = SVC(kernel='linear')  # Choose kernel type as appropriate
    clf.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f'Fold accuracy: {accuracy * 100:.2f}%')
# Print average cross-validation accuracy
print(f'Cross-Validation Accuracy:, {np.mean(accuracies) * 100:.2f}%')
print(data.shape)

Fold accuracy: 74.14%
Fold accuracy: 79.31%
Fold accuracy: 81.03%
Fold accuracy: 87.72%
Fold accuracy: 82.46%
Cross-Validation Accuracy:, 80.93%
(288, 12, 150)
